# AI Agents

In this assignment, your task is to implement ... The algorithms are a recap from your earlier Algorithms and Data Structures class, but the input is slightly different: the graph is not materialized. Instead, it is generated dynamically and you can access a single node of the graph at a time.

## Agent

We begin by considering a generic problem-solving agent. The agent receives *percepts* from its *environment* through *sensors* and uses *actuators* to perform *actions* to influence the environment. The process of converting the percepts to the actions (denoted by the question mark in the picture below) is the core function of the agent.

![](aima-images/fig2_1.png)

This picture is very general and covers a broad range of agents, both software (e.g., a chatbot) and hardware (e.g., a robot vacuum cleaner). For now, we will simplify. We will ignore sensors and actuators and concentrate only on the core. We will implement it according to the following contract represented as the class `Agent` (see cell below) with a single method: `do_step`. The sole argument of the method are the percepts of the agent and it should return an action to be executed next.

In [164]:
import numpy as np
from collections import deque
import copy
class Agent:
    def next_action(self, percepts):
        ...
        return action

## Problem

The environment above represents a real environment, meaning that any action in it will be visible to the user. Usually, it makes it unsuitable for planning what action to take. For example, if one plans to go on a car trip from Poznań to Berlin, one does not start driving blindly trying to find a route to Berlin. Instead, one takes a map and search for it on a model of the environment. We will combine this model with the definition of the problem to be solved, i.e., a starting position and a destination.

This is represented by the class `Problem` below. It has a single property `initial_state` that returns the starting position in the problem, e.g., the city of Poznań in the example about the trip. There are also three methods:
* `available_actions` returns, for a given `state` (e.g., a currently considered city), what actions are to be considered (e.g., what are the neighbouring cities one could go to directly from the currently considered city)
* `do_action` returns new state reached from the state `state` by executing the action `action` (e.g., the state representing that begin in Poznań and taking the highway to Warsaw, one ends up in Warsaw)
* `is_goal` which returns `True` if `state` is (one of) the destination(s) for the agent in this particular problem.

In [165]:
class Problem:
    @property
    def initial_state(self):
        ...
        
    def available_actions(self, state):
        ...        
        
    def do_action(self, state, action):
        ...
        return new_state
    
    def is_goal(self, state) -> bool:
        ...

To ground the problem definition in something more concrete consider a small flat of two dirty rooms and a robotic vacuum cleaner, as depicted in the following picture:
![](aima-images/fig2_2.png)

In the model, there are two rooms (A, B), each of them is either dirty or clean and the robot can always go to the left or to the right (maybe bumping into a wall), or suck the dirt (even if there's no dirt).

This is a model. Maybe the real environment is similar: two very small, rectangular rooms such that a single "suck" of the robot cleans the whole room at a time. Maybe it is very dissimilary: there are more rooms and they're of varying shapes, and actually cleaning them is a tedious task. The agent doesn't know and must thus assume that the `Problem` represents the environment well enough.

Assume the initial state looks like the picture above, and there are two goal states: both rooms clean and robot in either room A or room B. Lets formalize all this into the class `VacuumProblem`

In [166]:
class VacuumProblem(Problem):
    @property
    def initial_state(self):
        return (0, (True, True))
    
    def available_actions(self, state):
        return ["Left", "Suck", "Right"]
        
    def do_action(self, state, action):
        robot, dirty = state
        if action == "Left":
            return (max(robot-1, 0), dirty)
        elif action == "Suck":
            new_dirty = list(dirty)
            new_dirty[robot] = False
            return (robot, tuple(new_dirty))
        elif action == "Right":
            return (min(robot+1, len(dirty)-1), dirty)        
        raise Exception('Invalid action')
    
    def is_goal(self, state) -> bool:
        return not any(state[1])

Lets test it a bit:

In [167]:
problem = VacuumProblem()
state = problem.initial_state
print(state)
state = problem.do_action(state, 'Right')

print("State after going right from the initial state", state)
print("Are we there yet?", problem.is_goal(state))

(0, (True, True))
State after going right from the initial state (1, (True, True))
Are we there yet? False


In [168]:
state = problem.do_action(state, 'Right')

print("State after going right again", state)
print("Are we there yet?", problem.is_goal(state))

State after going right again (1, (True, True))
Are we there yet? False


In [169]:
state = problem.do_action(state, 'Suck')

print("State after sucking", state)
print("Are we there yet?", problem.is_goal(state))

State after sucking (1, (True, False))
Are we there yet? False


In [170]:
state = problem.do_action(state, 'Left')
state = problem.do_action(state, 'Left')
state = problem.do_action(state, 'Suck')

print("State after going to the left twice and sucking", state)
print("Are we there yet?", problem.is_goal(state))

State after going to the left twice and sucking (0, (False, False))
Are we there yet? True


## Oracular agent

Consider the following oracular agent. It is initialized with a problem and has an embedded oracle, which provides it with a sequence of actions to be executed to reach a goal in the problem.

In [171]:
class OracularAgent(Agent):
    def __init__(self, problem):
        self.problem = problem
        self.plan = self.oracle()
        
    def next_action(self, percepts):
        return self.plan.pop(0)
    
    def oracle(self):
        return ['Suck', 'Right', 'Suck']

Now it is the time to test this agent. We create an instance of the `VacuumProblem` and then ask the agent for `next_action`s until goal is not reach.

In [172]:
problem = VacuumProblem()
agent = OracularAgent(problem)

state = problem.initial_state
while not problem.is_goal(state):
    action = agent.next_action(None)
    state = problem.do_action(state, action)
    
print("Reached state:", state)
print("Is it goal?", problem.is_goal(state))

Reached state: (1, (False, False))
Is it goal? True


Observe that the agent works remarkably well and is able to reach a goal state with no problem whatsoever. It is fast, the code is short. There is only one obvious drawback: the solution is hard-coded and completely unsuitable not only to other types of problems, but also for other variants of the vacuum problem, e.g., such that the robot must return to the room A. We must therefore look for a more general solution: a searching agent.

## Task 1: Implement a breadth-first searching agent

Complete the method `bfs` in the class `BFSAgent` in the next cell. It should implement breadth-first search, an algorithm you should be familar with after the *Algorithms and data structures* classes. The main difference is that the graph is not explicitly specified not available as a whole. Instead, you must look on the `Problem` as a graph generator: a state corresponds to a node in the graph and each action available in the state corresponds to an arc from the current state to the state reached by executing this action. The following picture should help you to understand the concept.

![](aima-images/fig3_3.png)

In [173]:
def solution(node):
    action_list = []
    while (node.parent is not None):
        action_list.append(node.action)
        node = node.parent
    return action_list[::-1]

class Node1():
    def __init__(self, problem, parent, action, state):
        self.problem = problem
        self.state = state
        self.parent = parent
        self.action = action

    def children(self):
        children_nodes = []
        for action in self.problem.available_actions(self.state):
            children_nodes.append(Node1(self.problem, parent=self, action=action, state=self.problem.do_action(self.state, action)))
        return children_nodes

class BFSAgent(Agent):
    def __init__(self, problem):
        self.problem = problem
        self.plan = self.bfs()
        
    def next_action(self, percepts):
        return self.plan.pop(0)
    
    def bfs(self):
        root_node = Node1(self.problem, parent = None, action=None, state=problem.initial_state)
        
        frontier = deque()
        explored = set()
        frontier_check = set()

        frontier.append(root_node)
        frontier_check.add(root_node.state)

        while(True):
            if len(frontier) == 0:
                return [], 0 

            node = frontier.popleft()
            frontier_check.discard(node.state)
            if node.state not in explored or child.state not in frontier_check:
                if self.problem.is_goal(node.state):
                    return solution(node)
            explored.add(node.state)
            
            children = node.children()
            for child in children:
                frontier.append(child)
                frontier_check.add(child.state)
        

problem = VacuumProblem()
agent = BFSAgent(problem)
print("Plan to be executed", agent.plan)

Plan to be executed ['Suck', 'Right', 'Suck']


Lets test the solution in the same framework as the oracular agent

In [174]:
problem = VacuumProblem()
agent = BFSAgent(problem)

print("Plan to be executed", agent.plan)

state = problem.initial_state
while not problem.is_goal(state):
    action = agent.next_action(None)
    state = problem.do_action(state, action)
    
print("Reached state:", state)
print("Is it goal?", problem.is_goal(state))

Plan to be executed ['Suck', 'Right', 'Suck']
Reached state: (1, (False, False))
Is it goal? True


## Task 2: Implementing depth-first searching agent

Complete the method `dfs` in the class `BFSAgent` in the next cell. This time the task is to implement depth-first search (DFS). Be careful: DFS is vulnerable to infinite loops. Implement some sort of protection against them.

In [175]:
class Node2():
    def __init__(self, problem, state, parent, action):
        self.problem = problem
        self.state = state
        self.parent = parent
        self.action = action

    def return_child(self, action):
        child = Node2(self.problem, state=self.problem.do_action(self.state, action=action), parent=self, action=action)
        return child


class DFSAgent:
    def __init__(self, problem):
        self.problem = problem
        self.plan = solution(node=self.iter_dfs())

    def next_action(self, percepts):
        try:
            return self.plan.pop(0)
        except IndexError:
            return 'aha'
        
    # algorithm implemneted as shown in the "Artificial Intelligence: A Modern Approach" 3rd ed.
    def recursive_dfs(self, node, limit):
        if self.problem.is_goal(node.state):
            return node
        elif limit == 0:
            return "cutoff"
        else:
            cutoff_occurred = False
            for action in self.problem.available_actions(node.state):
                child = node.return_child(action=action)
                result = self.recursive_dfs(child, limit - 1)
                if result == "cutoff":
                    cutoff_occured = True
                elif result != "failure":
                    return result
            if cutoff_occured:
                return "cutoff"
            else:
                return "failure"

    def dfs(self, limit):
        return self.recursive_dfs(Node2(self.problem, state=self.problem.initial_state, parent=None, action=None),
                                  limit)

    def iter_dfs(self):
        for x in range(0, 100000000): #(0, infinity)
            result = self.dfs(limit=x)
            if result != "cutoff":
                return result
            
problem = VacuumProblem()
agent = DFSAgent(problem)
print("Plan to be executed", agent.plan)

Plan to be executed ['Suck', 'Right', 'Suck']


In [176]:
state = problem.initial_state
while not problem.is_goal(state):
    action = agent.next_action(None)    
    state = problem.do_action(state, action)
    
print("Reached state:", state)
print("Is it goal?", problem.is_goal(state))

Reached state: (1, (False, False))
Is it goal? True


## Task 3: Test it on 8-puzzle

The 8-puzzle, an instance of which is shown in the figure, consists of a 3×3 board with
eight numbered tiles and a blank space. A tile adjacent to the blank space can slide into the space. The object is to reach a specified goal state, such as the one shown on the right of the figure. The standard formulation is as follows:

* *States*: A state description specifies the location of each of the eight tiles and the blank in one of the nine squares.
* *Initial state*: Any state can be designated as the initial state. Note that any given goal can be reached from exactly half of the possible initial states.
* *Actions*: The simplest formulation defines the actions as movements of the blank space Left, Right, Up, or Down. Different subsets of these are possible depending on where the blank is.
* *Transition model*: Given a state and action, this returns the resulting state; for example, if we apply Left to the start state in the figure, the resulting state has the 5 and the blank switched.
* *Goal test*: This checks whether the state matches the goal configuration shown in the figure. (Other goal configurations are possible.)

![](aima-images/fig3_4.png)

Your task is to implement `PuzzleProblem` as a subclass of the class `Problem` to provide a formal problem description of the 8-puzzle problem described above. Then test your `DFSAgent` and `BFSAgent` on this new problem. 

It's fine if your agents are incapable of solving the problem for the presented start state and goal due to the excessive length of the necessary plan. If this the case, simplify the problem by changing the start state and/or the goal. The idea of this task is to show that your implementations are correct and the agents are capable - in principle - of solving this problem, not waiting for hours on end for computations to complete.

In [177]:
class PuzzleProblem():
    def __init__(self, init_state, goal_state):
        self.init_state = init_state
        self.goal_state = goal_state

    # Agent should work on immutables
    def convert_to_tuple(self, state):
        this_tuple = tuple(list([tuple(list(state[0])), tuple(list(state[1])), tuple(list(state[2]))]))
        return this_tuple

    def convert_to_np_array(self, state):
        a_list = []
        for x in state:
            a_list.append(list(x))
        array = np.array(a_list)
        return array

    @property
    def initial_state(self):
        return self.convert_to_tuple(self.init_state)

    def get_position_of_0(self, state):
        position = np.where(state == 0)
        return position[0][0], position[1][0]

    def available_actions(self, state):
        state = self.convert_to_np_array(state)
        
        a, b = self.get_position_of_0(state)
        available_actions_ = list()
        if b != 0:
            available_actions_.append('Left')
        if b != 2:
            available_actions_.append('Right')
        if a != 0:
            available_actions_.append('Up')
        if a != 2:
            available_actions_.append('Down')
        
        return available_actions_

    def do_action(self, state, action):
        state = self.convert_to_np_array(state)
        
        a, b = self.get_position_of_0(state)
        new_state = np.copy(state)
        if action == 'Left':
            if b == 0:
                raise Exception('Invalid action.')
            new_state[a][b], new_state[a][b - 1] = new_state[a][b - 1], new_state[a][b]
        elif action == 'Right':
            if b == 2:
                raise Exception('Invalid action.')
            new_state[a][b], new_state[a][b + 1] = new_state[a][b + 1], new_state[a][b]
        elif action == 'Up':
            if a == 0:
                raise Exception('Invalid action.')
            new_state[a][b], new_state[a - 1][b] = new_state[a - 1][b], new_state[a][b]
        elif action == 'Down':
            if a == 2:
                raise Exception('Invalid action.')
            new_state[a][b], new_state[a + 1][b] = new_state[a + 1][b], new_state[a][b]
        else:
            raise Exception('Invalid action.')
        
        new_state = self.convert_to_tuple(new_state)
        return new_state

    def is_goal(self, state) -> bool:
        state = self.convert_to_np_array(state)
        return (state == self.goal_state).all()


In [178]:
problem = PuzzleProblem(np.array([[3,2,5], [4,1,0], [6,7,8]]), np.array([[0,1,2],[3,4,5], [6,7,8]]))
agent = BFSAgent(problem)

print("Plan to be executed", agent.plan)

state = problem.initial_state
while not problem.is_goal(state):
    action = agent.next_action(None)
    state = problem.do_action(state, action)
    print(problem.convert_to_np_array(state))
    print('-----------')
    
print("Reached state:", state)
print("Is it goal?", problem.is_goal(state))

Plan to be executed ['Up', 'Left', 'Down', 'Left', 'Up']
[[3 2 0]
 [4 1 5]
 [6 7 8]]
-----------
[[3 0 2]
 [4 1 5]
 [6 7 8]]
-----------
[[3 1 2]
 [4 0 5]
 [6 7 8]]
-----------
[[3 1 2]
 [0 4 5]
 [6 7 8]]
-----------
[[0 1 2]
 [3 4 5]
 [6 7 8]]
-----------
Reached state: ((0, 1, 2), (3, 4, 5), (6, 7, 8))
Is it goal? True


In [179]:
problem = PuzzleProblem(np.array([[3,2,5], [4,1,0], [6,7,8]]), np.array([[0,1,2],[3,4,5], [6,7,8]]))
agent = DFSAgent(problem)

print("Plan to be executed", agent.plan)

state = problem.initial_state
while not problem.is_goal(state):
    action = agent.next_action(None)
    state = problem.do_action(state, action)
    print(problem.convert_to_np_array(state))
    print('-----------')
    
print("Reached state:", state)
print("Is it goal?", problem.is_goal(state))

Plan to be executed ['Up', 'Left', 'Down', 'Left', 'Up']
[[3 2 0]
 [4 1 5]
 [6 7 8]]
-----------
[[3 0 2]
 [4 1 5]
 [6 7 8]]
-----------
[[3 1 2]
 [4 0 5]
 [6 7 8]]
-----------
[[3 1 2]
 [0 4 5]
 [6 7 8]]
-----------
[[0 1 2]
 [3 4 5]
 [6 7 8]]
-----------
Reached state: ((0, 1, 2), (3, 4, 5), (6, 7, 8))
Is it goal? True


------------
The pictures and the description of 8-puzzle are from "Artificial Intelligence: A Modern Approach" 3rd ed.